# K-Nearest Neighbors with Ship Data
**Before you begin.**
You will edit this Colab notebook by adding/changing code and adding text to answer questions and provide explanations and insights.  When you are finished, you can print to PDF in order to easily generate a report.  To earn full points,  
*   try your best to avoid code that extends past the vertical bar in code cells.  It will not print well.
*   print text along with the answer/explanation, if answering/explaining with code.  For example, use `print(f'The bias is {bias}.')` instead of `print(bias)`.  This way, your instructor knows that you know how to interpret what you are seeing,
*   use italicized text if answering/explaining with text.  This will help your answers stand out, and
*   identify and explain some key observations that were not explcitily asked for in this assignment.  Sometimes the instructor wants to see if you know what to look for on your own.

You can add cells of either text or code by hovering over the horizontal edge of the cell just before where you want to add a new cell.  Click either "+ text" or "+ code", as appopriate.  You may have to click in the previous cell in order to see "+ text" or "+ code"


---



**Assignment summary.** This assignment will have you investigate the ability of the K-nearest neighbors algorithm (KNN) to classify images in the [San Francisco ship aerial imagery dataset](https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery) as ship or no-ship.  As opposed to using the black-box function, you will code KNN from scratch and observe how the choice of K affects classification results.  Since the dimension of the dataset is too high for KNN, and having recently studied principal component analysis (PCA), you will feed a PCA-reduced dataset to KNN and then rigorously test the performance with model assessment techniques like train-test split, confusion matrix, Matthews correlation coefficient, etc.  Thus this assignment is the entire ML pipeline: preprocessing (with PCA) --> classification (with KNN) --> model assessment. 

The black and white ship dataset is the same in this assignment as in the previous assignments.  

---


**Import the data.** Connect this Colab notebook to your Google Drive.  This code is complete.

In [ ]:
# Given your permission, this will connect your notebook to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Load in the ship data from Google Drive and print out the number of images.  The name of the folder containing all the ships should be `shipdata_MLcourse`, otherwise you need to change the code below accordingly.  The variable `img_data` contains all of the pixel information for each of the images.  This code is complete.

In [ ]:
# Now the ship data can be loaded into the notebook (presuming you uploaded..
# ...the data previously into your Google Drive)
import os, pickle

my_path = '/content/drive/My Drive/shipdata_MLcourse'
with open(os.path.join(my_path, 'img_data.pickle'), 'rb') as handle:
  img_data = pickle.load(handle)

print(f'The number of images is {len(img_data)}.') # print the number of images
print(f'Each image is a vector of length {len(img_data[0])}.')
print(f'The shape of img_data is {img_data.shape}.')          

Get the targets vector `y`: 1 if the image contains a ship, 0 for no ship (or partial ship). This code is complete.


In [ ]:
with open(os.path.join(my_path, 'targets.pickle'), 'rb') as handle:
  y = pickle.load(handle)
  print(f'The length of y is {len(y)}.')

Check out some of the ship images!  Each image is a vector of length 6400 that can be reshaped into 80 by 80 to make a small picture.  Change the value of `my_index` to view different images.  Remember that Python indexing starts at zero.  The code will print out the target as well.  Images towards the end of the dataset (larger indices) contain the ships. The code is complete.

In [ ]:
from google.colab.patches import cv2_imshow

my_index = 1000        # change this number
img = img_data[my_index].reshape((80,80))
cv2_imshow(img)  # display image

if y[my_index] == 0:
  print('This image is no ship or partial ship.')
else:
  print('This image is ship.')

**Use PCA to reduce the dimension of the ship data.** We will reduce the dimension of the dataset since 6400 is too large.  To keep this part simple, we will use the black-box function [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) and reduce the dimension down to 2 dimensions (`n_components`) and plot the reduced vectors in feature space.  Later, we will experiment with the number of dimensions to see how it affects the classification results.  The following code is complete.  Observe the plot.  How good do you think the classification results will be?  Why? 

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

n_components = 2 

pca = PCA(n_components=n_components)
X = pca.fit_transform(img_data)

# ----can only make a plot if in two dimensions----
if n_components == 2:
  plt.scatter(X[np.where(y==0),0], X[np.where(y==0),1], color='r')
  plt.scatter(X[np.where(y==1),0], X[np.where(y==1),1], color='b')
  plt.legend(('no ship', 'ship'))
  plt.xlabel('PC1')
  plt.ylabel('PC2')
  plt.title('Dimensionally Reduced Ship Data')

**Next, we will train-test the KNN algorithm using the dimensionally reduced data.**  To keep the testing simple, we will randomly split the data once using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), instead of many times with cross-validation.  Use `train_test_split` to randomly split the data into training and testing partitions so the test set is 25% of the entire dataset.  Give the function a `random_state` of 0.  The data training set, data testing set, training labels, and testing labels should be `Xtrn`, `Xtst`, `ytrn`, and `ytst`, respectively.

In [ ]:
from sklearn.model_selection import train_test_split
# TODO: Randomly split the data into training and testing
Xtrn,Xtst,ytrn,ytst=

Scale the data to zero mean and unit variance.  Use [StandarScalar](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) to normalize both the training data `Xtrn` and the testing data `Xtst`. Recall that the test set should be normalized with the same parameters as the training set.  (That is, you will only fit `StandardScaler` once.) 

In [ ]:
from sklearn.preprocessing import StandardScaler
# TODO: Scale the data.  Use StandardScaler.





With K = 3, code the (from-scratch) KNN algorithm.  The easiest and most straightforward way to code the algorithm is the following:
1.   Initialize the test target predictions as an array of zeros of the appropriate length.
2.   Loop through each vector in the normalized test set.  Use `enumerate` to return the index with each vector.  For each vector, use Python broadcasting to make an array of the vector difference of each vector in the reduced training set with the test vector.  Use the `norm` function from the `linalg` sub-library of `numpy` to compute the distance away the test vector is from every training vector. 
1.   Use the [argpartition](https://stackoverflow.com/questions/34226400/find-the-index-of-the-k-smallest-values-of-a-numpy-array) function from `numpy` to return the indices of the smallest K distances.
2.   With the K indices from the previous step, establish a logical test to determine if the majority of the K training labels are of class 1.  If so, set the test prediction to 1, otherwise do nothing (since you initialized the prediction to 0 in Step 1.).

When the code is finished, you should have a vector of 1's and 0's corresponding to the prediction of each test point in the test set.  Note that the variable K is a hyperparmeter, and you will later investigate how its value affects the results.  So, do not hardcode its value as 3. Instead, use the variable `K`.  

You will not receive credit if you use the black-box functions from `sklearn`.

In [ ]:
K = 3
# TODO: Code the KNN algorithm from scratch. It should take about 6 lines...






How did it do?  Compare the test predictions with the actual test targest.  Make a convincing argument. 

In [ ]:
# TODO: Analyze the performance of the KNN algorithm














**Go back and change the hyperparameters.** The entire algorithm has two hyperparameters: (1) the number of principal components to keep, and (2) the value of `K`. Go back and change the hyperparmeters to reasonable values (keep `K` as an odd number).  Try to find the optimal combination of hyperparameters.  How does the algorithm perform?  Describe the values with which you experiment and the corresponding algorithm performance.

**Go further.** In a previous assignment, you investigated the performance of a logistic regression classifier on this same dataset.  Compare the performance with that of KNN.  Make sure you make a fair comparison.  If not, account for discrepancies in your analysis.

Use the black-box function [KNeighborsClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) to classify the normalized dimensionally reduced dataset.  Compare the results with the results of your from-scratch algorithm.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
# TODO: train and test KNN.  Use KNeighborsClassifier.















